In [51]:
!pip install faiss-cpu


In [52]:
!pip install scikit-surprise


In [53]:
!pip install fastapi uvicorn

In [54]:
!pip install torch torchvision torchaudio


In [72]:
!python movie3.py


In [73]:
!pip install pyngrok



In [74]:
!ngrok authtoken 2tdGyJLXWMLTSjJGVDsufQT8zh7_2PqVSXpE8uupLNrRQMWGo



Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [76]:
from pyngrok import ngrok

# Kill any remaining ngrok sessions
ngrok.kill()

# Start a new tunnel for FastAPI (port 8000)
public_url = ngrok.connect(8000)
print("Public FastAPI URL:", public_url)


Public FastAPI URL: NgrokTunnel: "https://e87b-35-245-8-105.ngrok-free.app" -> "http://localhost:8000"


In [ ]:
!uvicorn movie3:app --host 0.0.0.0 --port 8000 --reload &


INFO:     Will watch for changes in these directories: ['/content']
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
INFO:     Started reloader process [58511] using StatReload
INFO:     Started server process [58513]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     2603:8080:600:a79a:24ac:f1ef:5b78:cd5b:0 - "GET / HTTP/1.1" 200 OK
INFO:     2603:8080:600:a79a:24ac:f1ef:5b78:cd5b:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     2603:8080:600:a79a:24ac:f1ef:5b78:cd5b:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     2603:8080:600:a79a:24ac:f1ef:5b78:cd5b:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     2603:8080:600:a79a:24ac:f1ef:5b78:cd5b:0 - "POST /recommend HTTP/1.1" 200 OK
INFO:     2603:8080:600:a79a:24ac:f1ef:5b78:cd5b:0 - "POST /recommend HTTP/1.1" 200 OK


In [64]:
!tail -n 50 /content/movie1.py



def read_root():
    return {"message": "FastAPI is working!"}
    
# Request model for API
class RecommendationRequest(BaseModel):
    user_id: int
    top_k: int = 5

@app.post("/recommend")
def recommend_movies(request: RecommendationRequest):
    """
    API Endpoint to get enhanced movie recommendations based on:
    - LLM-based content understanding (BERT embeddings)
    - Sentiment analysis on user-generated tags
    - Collaborative filtering (SVD model from MovieLens)
    """
    user_id = request.user_id

    # Collaborative Filtering: Predict top-rated movies for user
    movie_ids = movies_df["movieId"].tolist()
    predictions = [(mid, collab_model.predict(user_id, mid).est) for mid in movie_ids]
    predictions.sort(key=lambda x: x[1], reverse=True)

    # Get top K collaborative filtering recommendations
    top_collab_movies = [pred[0] for pred in predictions[:request.top_k]]

    # Use FAISS for LLM-based content recommendations
    content_query_embedding = get_embeddi